In [1]:
# PLCB4

In [2]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import IndividualTable, QcVisualizer, HpoaTableBuilder
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.31


In [3]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [4]:
df = pd.read_excel("input/PLCB4_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,Comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Narrow mouth,Sparse hair,Narrow forehead,Opisthotonus,Central sleep apnea,Mandibular condyle hypoplasia,Mild global developmental delay,Hypoplasia of the corpus callosum,Postnatal growth retardation,Hypotonia
0,str,str,str,optional str,str,str,str,HGVS str,HGVS str,optional str,...,HP:0000160,HP:0008070,HP:0000341,HP:0002179,HP:0010536,HP:0007628,\,HP:0002079,HP:0008897,HP:0001252
1,PMID:23315542,Heterogeneity of mutational mechanisms and modes of inheritance in auriculocondylar syndrome,case 8,NaN,OMIM:620458,Auriculocondylar syndrome 2B,NM_001377142.1,"4,997-bp intragenic deletion in the PLCB4 gene","4,997-bp intragenic deletion in the PLCB4 gene",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)

Created encoders for 41 fields


In [6]:
individuals = encoder.get_individuals()

In [11]:
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          cohort_name="PLCB4",
                          transcript="NM_001377142.1",
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2",
                         gene_id="HGNC:9059", 
                         gene_symbol="PLCB4")

[INFO] encoding variant "c.624del"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001377142.1%3Ac.624del/NM_001377142.1?content-type=application%2Fjson
[INFO] encoding variant "c.854-1G>A"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001377142.1%3Ac.854-1G>A/NM_001377142.1?content-type=application%2Fjson
[INFO] encoding variant "c.1620dup"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001377142.1%3Ac.1620dup/NM_001377142.1?content-type=application%2Fjson
[INFO] encoding variant "c.1238+1G>C"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_001377142.1%3Ac.1238+1G>C/NM_001377142.1?content-type=application%2Fjson


In [12]:
vmanager.to_summary()

,status,count,alleles
0,mapped,4,"c.624del, c.854-1G>A, c.1620dup, c.1238+1G>C"
1,unmapped,1,"4,997-bp intragenic deletion in the PLCB4 gene"


In [13]:
deletion_set = {'4,997-bp intragenic deletion in the PLCB4 gene'}
vmanager.code_as_chromosomal_deletion(allele_set=deletion_set)

In [14]:
vmanager.add_variants_to_individuals(individuals)
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [15]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
case 8 (MALE; ),Auriculocondylar syndrome 2B (OMIM:620458),"4,997-bp intragenic deletion in the PLCB4 gene: chromosomal_deletion (SO:1000029)",Question mark ear (HP:0030022); Postauricular skin tag (HP:0004451); Micrognathia (HP:0000347); excluded: Preauricular skin tag (HP:0000384); excluded: Facial asymmetry (HP:0000324); excluded: Limbal dermoid (HP:0001140); excluded: Abnormality of the cervical spine (HP:0003319); excluded: Neonatal respiratory distress (HP:0002643); excluded: Omphalocele (HP:0001539)
Patient 1 (MALE; ),Auriculocondylar syndrome 2B (OMIM:620458),NM_001377142.1:c.854-1G>A (heterozygous) NM_001377142.1:c.1238+1G>C (heterozygous),P1D: Neonatal respiratory distress (HP:0002643)Question mark ear (HP:0030022); Micrognathia (HP:0000347); Full cheeks (HP:0000293); Long penis (HP:0000040); Autistic behavior (HP:0000729); excluded: Omphalocele (HP:0001539); excluded: Feeding difficulties (HP:0011968)
Patient 2 (MALE; ),Auriculocondylar syndrome 2B (OMIM:620458),NM_001377142.1:c.854-1G>A (heterozygous) NM_001377142.1:c.1238+1G>C (heterozygous),Question mark ear (HP:0030022); Micrognathia (HP:0000347); Full cheeks (HP:0000293); Long penis (HP:0000040); Chronic constipation (HP:0012450); Omphalocele (HP:0001539); Feeding difficulties (HP:0011968)
proband (MALE; ),Auriculocondylar syndrome 2B (OMIM:620458),NM_001377142.1:c.624del (homozygous),Question mark ear (HP:0030022); Postauricular skin tag (HP:0004451); Full cheeks (HP:0000293); Failure to thrive (HP:0001508); Darwin tubercle of helix (HP:0011261); Telecanthus (HP:0000506); Synophrys (HP:0000664); Narrow mouth (HP:0000160); Sparse hair (HP:0008070); Narrow forehead (HP:0000341); Opisthotonus (HP:0002179); Central sleep apnea (HP:0010536); Mandibular condyle hypoplasia (HP:0007628); Hypoplasia of the corpus callosum (HP:0002079); Postnatal growth retardation (HP:0008897); excluded: Preauricular skin tag (HP:0000384)
patient 23 (FEMALE; ),Auriculocondylar syndrome 2B (OMIM:620458),NM_001377142.1:c.1620dup (homozygous),Question mark ear (HP:0030022); Micrognathia (HP:0000347); Feeding difficulties (HP:0011968); Hypotonia (HP:0001252)


In [16]:
encoder.output_individuals_as_phenopackets(individuals)

We output 5 GA4GH phenopackets to the directory phenopackets


In [18]:
PMID="PMID:33539324"
builder = HpoaTableBuilder.from_individuals(individuals, created_by)
builder.congenital_onset("PMID:23315542",1,1).congenital_onset("PMID:23913798",2,2).congenital_onset("PMID:27007857",1,1).congenital_onset("PMID:35170830",1,1)
hpoa_creator = builder.autosomal_recessive("PMID:23913798").build()
df = hpoa_creator.get_dataframe()
df

We found a total of 27 unique HPO terms
Extracted disease: Auriculocondylar syndrome 2B (OMIM:620458)


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620458,Auriculocondylar syndrome 2B,HP:0030022,Question mark ear,,,1/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620458,Auriculocondylar syndrome 2B,HP:0004451,Postauricular skin tag,,,1/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620458,Auriculocondylar syndrome 2B,HP:0000347,Micrognathia,,,1/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620458,Auriculocondylar syndrome 2B,HP:0000384,Preauricular skin tag,,,0/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620458,Auriculocondylar syndrome 2B,HP:0000324,Facial asymmetry,,,0/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
5,OMIM:620458,Auriculocondylar syndrome 2B,HP:0001140,Limbal dermoid,,,0/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
6,OMIM:620458,Auriculocondylar syndrome 2B,HP:0003319,Abnormality of the cervical spine,,,0/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
7,OMIM:620458,Auriculocondylar syndrome 2B,HP:0002643,Neonatal respiratory distress,,,0/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
8,OMIM:620458,Auriculocondylar syndrome 2B,HP:0001539,Omphalocele,,,0/1,,,,,PMID:23315542,PCS,ORCID:0000-0002-0736-9199
9,OMIM:620458,Auriculocondylar syndrome 2B,HP:0030022,Question mark ear,,,2/2,,,,,PMID:23913798,PCS,ORCID:0000-0002-0736-9199


In [19]:
hpoa_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620458.tab
